# Imports

In [1]:
# use this command for installing the MMDBWriter package
# !pip install -U git+https://github.com/VimT/MaxMind-DB-Writer-python

In [2]:
import netaddr
import ipaddress
import maxminddb
import pandas as pd
from tqdm import tqdm
from io import StringIO
from netaddr import IPSet
from mmdb_writer import MMDBWriter

# load data 

In [3]:
with open("./data/cidrs.txt", "r") as file:
    text = file.read()

## Preprocess
## Remove redundancy, select only IP v4

In [4]:
cidrs = text.split("\n")
cidrs = [item for item in cidrs if len(item) > 0]
cidrs = pd.Series(cidrs).unique()
# only use version 4 cidrs
cidrs = [cidr for cidr in cidrs if ipaddress.ip_network(cidr).version == 4]

In [5]:
def remove_not_superne(cidrs_list):    
    remove_list = []
    # calc network for each cirdr, catch for speed up
    cidr2network = {}
    for cidr in cidrs_list:
        cidr2network[cidr] = ipaddress.ip_network(cidr)
    
    for i in tqdm(range(0, len(cidrs_list))):
        cird_i = cidrs_list[i]    
        if(cird_i in remove_list):
            continue
        network1 = cidr2network[cird_i]
        for j in range(i+1, len(cidrs_list)):        
            cird_j = cidrs_list[j]        
            network2 = cidr2network[cird_j]
            if network1.supernet_of(network2):                
                remove_list.append(cird_j)            
            elif network2.supernet_of(network1):                
                remove_list.append(cird_i)  
                break        
    return remove_list 

rm = remove_not_superne(cidrs)

100%|██████████| 1945/1945 [00:07<00:00, 261.77it/s]


In [6]:
print(len(cidrs))
print(len(rm))
cidrs = set(cidrs) - set(rm)
print(len(cidrs))

1945
0
1945


# MMDB Writer

In [7]:
writer = MMDBWriter()
for cidr in tqdm(cidrs):    
    writer.insert_network(IPSet([cidr]), {'country': 'IR', 'isp': "NA"})    
writer.to_db_file('./result/geoipIR.mmdb')

100%|██████████| 1945/1945 [00:00<00:00, 17506.91it/s]


#  Test dataste

In [9]:
db = maxminddb.open_database('./result/geoipIR.mmdb')

{'country': 'IR', 'isp': 'NA'}

In [10]:
db.get("79.127.127.16")

{'country': 'IR', 'isp': 'NA'}

In [14]:
db.get("172.67.73.163")